In [1]:
import os 
os.environ['XDG_SESSION_TYPE'] = 'x11'
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
from photogrammetry_ai import (
    PhotogrammetryPipeline,
    LightGlueMatcher,
    VGGTReconstructor,
    ICPAligner,
)
import os

image_dir = "/home/jourdelune/Images/colmap/input"

images = os.listdir(image_dir)
images = [
    os.path.join(image_dir, img)
    for img in images
    if img.lower().endswith((".jpg", ".jpeg", ".png"))
]


pipeline = PhotogrammetryPipeline(
    matcher=LightGlueMatcher(),  # used to create related batches of images
    reconstructor=VGGTReconstructor(),  # used to reconstruct the 3D points from the images
    aligner=ICPAligner(),  # used to merges the 3D points from the batches
    max_batch_size=4,
)

results = pipeline.process(images)

/home/jourdelune/Bureau/dev/photogrammetry_ai/.venv/lib/python3.12/site-packages/lightglue/lightglue.py:24: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/home/jourdelune/Bureau/dev/photogrammetry_ai/src/photogrammetry_ai/reconstruction/vggt.py:76: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(dtype=self.dtype):


:: RANSAC registration on downsampled point clouds.
   Since the downsampling voxel size is 0.050,
   we use a liberal distance threshold 0.075.
[Open3D WARNING] Too few correspondences (78) after mutual filter, fall back to original correspondences.
:: RANSAC registration on downsampled point clouds.
   Since the downsampling voxel size is 0.050,
   we use a liberal distance threshold 0.075.
:: RANSAC registration on downsampled point clouds.
   Since the downsampling voxel size is 0.050,
   we use a liberal distance threshold 0.075.
[Open3D WARNING] Too few correspondences (1114) after mutual filter, fall back to original correspondences.
:: RANSAC registration on downsampled point clouds.
   Since the downsampling voxel size is 0.050,
   we use a liberal distance threshold 0.075.
[Open3D WARNING] Too few correspondences (906) after mutual filter, fall back to original correspondences.
:: RANSAC registration on downsampled point clouds.
   Since the downsampling voxel size is 0.050,


In [3]:
del pipeline

o3d.visualization.draw_geometries([results])

: 